In [22]:
import redis
import cst_python as cst
import json

In [23]:
mind_name = "default_mind"

In [24]:
client = redis.Redis(decode_responses=True)
pubsub = client.pubsub()

In [25]:
client.flushall()

True

Node1 publica que existe

In [26]:
client.lpush(f"{mind_name}:nodes", "node1")

1

In [27]:
class MemoryEncoder(json.JSONEncoder):
    def default(self, memory:cst.core.entities.Memory):
        return MemoryEncoder.to_dict(memory)
    
    @staticmethod
    def to_dict(memory:cst.core.entities.Memory):
        data = {
            "timestamp": memory.get_timestamp(),
            "evaluation": memory.get_evaluation(),
            "I": memory.get_info(),
            "name": memory.get_name(),
            "id": memory.get_id()
        }

        return data

Node1 checa se memória com id "memory1" existe. Como não, publica key:

In [28]:
def update_memory(memory_name, memory_object:cst.MemoryObject, client:redis.Redis):
    timestamp = float(client.hget(f"{mind_name}:memories:{memory_name}", "timestamp"))
    
    if memory_object.timestamp < timestamp:
        print("Retrieve update")
        memory_dict = client.hgetall(f"{mind_name}:memories:{memory_name}")

        memory_object.set_evaluation(float(memory_dict["evaluation"]))
        memory_object.set_name(memory_dict["name"])
        memory_object.set_id(float(memory_dict["id"]))

        info_json = memory_dict["I"]
        info = json.loads(info_json)

        memory_object.set_info(info)

        memory_object.timestamp = float(memory_dict["timestamp"])
    elif memory_object.timestamp > timestamp:
        print("Send update")
        memory_dict = MemoryEncoder.to_dict(memory_object)
        memory_dict["I"] = json.dumps(memory_dict["I"])

        client.hset(f"{mind_name}:memories:{memory_name}", mapping=memory_dict)
        client.publish(f"{mind_name}:memories:{memory_name}:update", "")

In [29]:
def create_memory(node, memory_name, client:redis.Redis, pubsub:redis.client.PubSub) -> cst.MemoryObject:
    memory = cst.MemoryObject()

    if client.exists(f"{mind_name}:memories:{memory_name}"):
        memory_dict = client.hgetall(f"{mind_name}:memories:{memory_name}")

        if memory_dict["owner"] != "":
            #Solicita memória
            pass


        #Copia memória
        print("Copia")

        memory_dict = client.hgetall(f"{mind_name}:memories:{memory_name}")

        memory.set_evaluation(float(memory_dict["evaluation"]))
        memory.set_name(memory_dict["name"])
        memory.set_id(float(memory_dict["id"]))

        info_json = memory_dict["I"]
        info = json.loads(info_json)

        memory.set_info(info)

        memory.timestamp = float(memory_dict["timestamp"])
    else:
        #Indica que memória existe
        print("Cria")
        client.lpush(f"{node}:memories", memory_name)

        memory_dict = MemoryEncoder.to_dict(memory)
        memory_dict["I"] = json.dumps(memory_dict["I"])
        memory_dict["owner"] = "" #node

        client.hset(f"{mind_name}:memories:{memory_name}", mapping=memory_dict)

    subscribe_func = lambda message : update_memory(memory_name, memory, client)
    pubsub.subscribe(**{f"{mind_name}:memories:{memory_name}:update":subscribe_func})

    return memory

In [30]:
memory1 = create_memory("node1", "memory1", client, pubsub)

Cria


In [31]:
memory1

MemoryObject [idmemoryobject=0.0, timestamp=0.0, evaluation=0.0, I=None, name=]

node2 entra no jogo

In [32]:
client2 = redis.Redis(decode_responses=True)
pubsub2 = client2.pubsub()

In [33]:
client2.lpush(f"{mind_name}:nodes", "node2")

2

In [34]:
nodes = client2.lrange(f"{mind_name}:nodes", 0, -1)
nodes

['node2', 'node1']

node2 tenta criar memória, percebe que existe e sincroniza

In [35]:
client.hgetall(f"{mind_name}:memories:memory1")

{'timestamp': '0.0',
 'evaluation': '0.0',
 'I': 'null',
 'name': '',
 'id': '0.0',
 'owner': ''}

In [36]:
node2_memory1 = create_memory("node2", "memory1", client2, pubsub2)

node2_memory1

Copia


MemoryObject [idmemoryobject=0.0, timestamp=0.0, evaluation=0.0, I=None, name=]

In [37]:
memory2 = create_memory("node2", "memory2", client2, pubsub2)

memory2

Cria


MemoryObject [idmemoryobject=0.0, timestamp=0.0, evaluation=0.0, I=None, name=]

In [38]:
node2_memory1.set_info("INFO")
update_memory("memory1", node2_memory1, client2)

Send update


In [39]:
for _pubsub in [pubsub, pubsub2]:
    msg = _pubsub.get_message()
    while msg is not None:
        print(msg)
        msg = _pubsub.get_message()

{'type': 'subscribe', 'pattern': None, 'channel': 'default_mind:memories:memory1:update', 'data': 1}
Retrieve update
{'type': 'subscribe', 'pattern': None, 'channel': 'default_mind:memories:memory1:update', 'data': 1}
{'type': 'subscribe', 'pattern': None, 'channel': 'default_mind:memories:memory2:update', 'data': 2}


In [40]:
client.hgetall(f"{mind_name}:memories:memory1")

{'timestamp': '1725638895.8791993',
 'evaluation': '0.0',
 'I': '"INFO"',
 'name': '',
 'id': '0.0',
 'owner': ''}